In [182]:
import yfinance as yf
import numpy as np
import pandas as pd
import datetime as dt
import talib
import matplotlib.pyplot as plt

from skfolio.population import Population
from skfolio.portfolio import Portfolio
import warnings
warnings.filterwarnings('ignore')

In [219]:
tickers = ['INFRABEES', 'CPSEETF']
tickers = [t  + '.NS' for t in tickers]

end = dt.date.today()
start = end - dt.timedelta(5 * 365)

df = yf.download(tickers, 
                 interval='1d',
                 period='max', 
                 auto_adjust=True, 
                 group_by='ticker').dropna()
df

[*********************100%%**********************]  2 of 2 completed


Ticker     INFRABEES.NS                                                \
Price              Open        High         Low       Close    Volume   
Date                                                                    
2018-05-11   345.000000  346.799988  344.000000  344.850006     279.0   
2018-05-18   337.000000  337.549988  331.700012  332.660004     162.0   
2018-05-25   336.000000  337.600006  336.000000  336.049988      81.0   
2018-06-01   340.500000  340.500000  335.399994  337.890015     166.0   
2018-06-08   331.500000  333.549988  329.250000  331.920013     313.0   
...                 ...         ...         ...         ...       ...   
2024-08-09   952.380005  959.000000  943.109985  947.130005  484291.0   
2024-08-12   947.130005  949.820007  938.739990  945.469971   16073.0   
2024-08-13   945.469971  947.260010  937.000000  938.950012   12115.0   
2024-08-14   938.960022  941.119995  932.989990  935.719971    9973.0   
2024-08-16   941.000000  950.000000  935.169983  949.429993    8748.0   

Ticker      CPSEETF.NS                                               
Price             Open        High         Low       Close   Volume  
Date                                                                 
2018-05-11   28.059999   28.290001   28.049999   28.150000   593373  
2018-05-18   27.639999   27.940001   27.270000   27.290001   257763  
2018-05-25   26.299999   26.990000   26.200001   26.930000   323307  
2018-06-01   28.480000   28.480000   27.500000   27.620001   330232  
2018-06-08   27.660000   27.820000   27.510000   27.760000   565940  
...                ...         ...         ...         ...      ...  
2024-08-09  102.889999  103.980003  102.500000  103.220001  6881415  
2024-08-12  104.750000  104.750000  101.910004  102.900002  5104440  
2024-08-13  103.790001  103.790001  101.550003  101.769997  3091846  
2024-08-14  102.199997  102.610001  100.250000  100.440002  4591516  
2024-08-16  101.739998  102.000000  100.500000  101.839996  3368980  

[1290 rows x 10 columns]

In [220]:
df1 = df.copy()
d = dict()

for t in tickers:
    d[t] = df1[t]
    d[t]['RSI'] = talib.RSI(d[t]['Close'].to_numpy())
    d[t]['ret'] = (1+d[t]['Close'].pct_change().dropna())
    d[t] = d[t].dropna()

In [223]:
merge_index = pd.merge(d[tickers[0]], d[tickers[1]], on='Date').index
for t in tickers:
    d[t] = d[t].loc[merge_index]

In [230]:
df2 = pd.DataFrame(d[tickers[0]].RSI > d[tickers[1]].RSI)
df2 = df2.dropna()
df2

,RSI
Date,
2018-08-17,False
2018-08-24,False
2018-08-31,False
2018-09-07,False
2018-09-14,True
...,...
2024-08-09,False
2024-08-12,False
2024-08-13,False


In [231]:
for i,t in enumerate(tickers):
    df2 = pd.merge(df2, d[t][['ret']], on='Date').rename(columns={'ret': f'ret{i}'})
df2

,RSI,ret0,ret1
Date,,,
2018-08-17,False,0.994616,0.991189
2018-08-24,False,1.034885,1.016667
2018-08-31,False,1.024654,1.008379
2018-09-07,False,0.988119,0.980491
2018-09-14,True,1.003021,0.989683
...,...,...,...
2024-08-09,False,1.005286,1.017046
2024-08-12,False,0.998247,0.996900
2024-08-13,False,0.993104,0.989018


In [232]:
def foo(row):
    if row['RSI']:
        return row['ret0']
    else:
        return row['ret1']

In [233]:
from skfolio.portfolio import BasePortfolio

strat = df2.apply(foo, axis=1)-1
portfolios = [BasePortfolio(returns=strat, observations=strat.index, name='strategy')]

for t in tickers:
    tmp = (d[t]['ret'] - 1)
    portfolios.append(BasePortfolio(returns=tmp, observations=tmp.index, name=t))

pop = Population(portfolios)
pop.set_portfolio_params(compounded=True)
pop.plot_cumulative_returns(log_scale=True)

In [229]:
pop.summary()

,strategy,INFRABEES.NS,CPSEETF.NS
Mean,0.27%,0.095%,0.12%
Annualized Mean,68.77%,24.04%,29.62%
Variance,0.025%,0.019%,0.028%
Annualized Variance,6.23%,4.78%,7.14%
Semi-Variance,0.012%,0.011%,0.014%
Annualized Semi-Variance,2.92%,2.69%,3.60%
Standard Deviation,1.57%,1.38%,1.68%
Annualized Standard Deviation,24.96%,21.86%,26.72%
Semi-Deviation,1.08%,1.03%,1.19%
Annualized Semi-Deviation,17.09%,16.39%,18.97%
